In [1]:
#导包
from netCDF4 import Dataset
from tensorflow.keras import layers, models
from tensorflow.keras.layers import ConvLSTM2D,LSTM, BatchNormalization, LayerNormalization,Input, Conv3D, TimeDistributed, Flatten
from tensorflow.keras.layers import Concatenate,Conv2D,TimeDistributed, MaxPooling2D, Input, MaxPooling3D
from tensorflow.keras.layers import  Reshape,multiply
from tensorflow.keras.layers import Layer,Lambda,Dot,ReLU, Dense, Dropout, Activation, Flatten,Attention
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.optimizers import Adam,SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.losses import MeanSquaredError
from sklearn.metrics import r2_score,accuracy_score,precision_score
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras import regularizers
import tensorflow as tf
from datetime import datetime
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import models, layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import tensorflow.keras as keras
from tensorflow.keras import layers, Sequential, regularizers
from tensorflow.keras.utils import  plot_model
# calculate RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas as pd 
import gc 
import gzip
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import match
import xarray as xr
import os 
%matplotlib inline

In [2]:
len(tf.config.experimental.list_physical_devices('GPU'))

1

# 读取数据

## SSH

In [3]:
data_ssh = nc.Dataset(r'D:/data/SSH/SSH_2005-2015_month.nc')

时间长度 132


In [4]:
time_ssh = data_ssh['time'][:]
print(len(time_ssh))
ssh_lat = data_ssh['lat'][:].data
ssh_lon = data_ssh['lon'][:].data
ssh = data_ssh['ssh'][:].data
print(ssh.shape)

In [5]:
print(np.nanmax(ssh),np.nanmin(ssh))

(132, 280, 600)
2.20051 -0.43673334


## SSS

In [6]:
data_sss = nc.Dataset(r'D:/data/SSS/SSS_2005-2020_month.nc')

In [7]:
time_sss = data_sss['time'][:132]
print(len(time_sss))
sss_lat = data_sss['lat'][:].data
sss_lon = data_sss['lon'][:].data
sss = data_sss['sos'][:132].data 
print(sss.shape)
sss = np.squeeze(sss)
print(sss.shape)

132


In [10]:
print('最大值与最小值:',np.nanmax(sss),np.nanmin(sss))

最大值与最小值: 36.072178 13.604756


## SSW

In [11]:
data_ssw = nc.Dataset(r'D:/data/SSW/SSW_2005-2018_month.nc')

132


In [12]:
time_ssw = data_ssw['time'][:132]
print(len(time_ssw))
lat_ssw = data_ssw['lat'][:].data
lon_ssw = data_ssw['lon'][:].data

In [13]:
uwnd = data_ssw['uwnd'][:132,314:594,440:1040].data  
vwnd = data_ssw['vwnd'][:132,314:594,440:1040].data
print(uwnd.shape)
print(vwnd.shape)

(132, 280, 600)
(132, 280, 600)


In [14]:
np.nanmax(uwnd),np.nanmin(uwnd),np.nanmax(vwnd),np.nanmin(vwnd)

(12.433594, -11.7988205, 8.8359995, -13.645481)

## SST 海表面温度

In [15]:
data_sst = nc.Dataset('D:/data/SST/SST_2005-2015_month.nc','r')

In [16]:
time_sst = data_sst['time'][:]
print(len(time_sst))
sst_lat = data_sst['lat'][:].data
sst_lon = data_sst['lon'][:].data
sst = data_sst['sst'][:].data
print(ssh.shape)

132


In [18]:
np.nanmax(sst),np.nanmin(sst)

(31.901611, -1.8474731)

## 3DS

In [19]:
data_3ds = nc.Dataset(r'D:/data/3DTS/3DS_2005-2015_month.nc')
sali_depth = data_3ds['so'][:].data
print(sali_depth.shape)

(132, 43, 280, 600)


In [20]:
depths = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
         450,500,550,600,700,800,900,1000,1100,1200,1300,1400,1500,1750,2000]
depths_use = [5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,500,600,700,800,900,1000,1100,
              1300,1500,1750,2000]

In [21]:
sali_depth_use = []
selected_layers = []
for i, depth in enumerate(depths):
    if depth in depths_use:
        selected_layers.append(sali_depth[:, i:i+1, :, :])
if selected_layers:
    sali_depth_use = np.concatenate(selected_layers, axis=1)
else:
    sali_depth_use = None

In [22]:
print(sali_depth_use.shape)
sali_depth_use = np.transpose(sali_depth_use,(0,2,3,1))
print(sali_depth_use.shape)

(132, 33, 280, 600)
(132, 280, 600, 33)


## 数据合并

In [23]:
print(ssh.shape,sst.shape,uwnd.shape,vwnd.shape,sss.shape)
cat_data =np.stack((ssh,sst,uwnd,vwnd,sss),axis=3)
cat_data.shape

(132, 280, 600) (132, 280, 600) (132, 280, 600) (132, 280, 600) (132, 280, 600)


(132, 280, 600, 5)

In [24]:
cat_surface_depth = np.concatenate((cat_data,sali_depth_use),axis=-1)
print(cat_surface_depth.shape)

(132, 280, 600, 38)


## 去掉合并数据中的nan


In [26]:
cat_surface_depth_reshape  = np.transpose(cat_surface_depth,(1,2,0,3))
print(cat_surface_depth_reshape.shape)
cat_surface_depth_reshape = cat_surface_depth_reshape.reshape ((-1,132*38))
print(cat_surface_depth_reshape.shape)

(280, 600, 132, 38)
(168000, 5016)


In [27]:
cat_surface_depth_reshape[cat_surface_depth_reshape == 32767] = np.nan
print('cat_surface_depth_reshape删除nan值之前',cat_surface_depth_reshape.shape)
nan_mask = np.isnan(cat_surface_depth_reshape)
cat_surface_depth_nonan = cat_surface_depth_reshape[~nan_mask.any(axis=1)]   
print('cat_surface_depth_reshape删除nan值之后',cat_surface_depth_nonan.shape)

cat_surface_depth_reshape删除nan值之前 (168000, 5016)
cat_surface_depth_reshape删除nan值之后 (104457, 5016)


In [28]:
cat_surface_depth_nonan = cat_surface_depth_nonan.reshape(-1,132,38)
print(cat_surface_depth_nonan.shape)
cat_surface_depth_nonan = np.transpose(cat_surface_depth_nonan,(1,0,2))
print(cat_surface_depth_nonan.shape)

(104457, 132, 38)
(132, 104457, 38)


## 将数据进行堆叠

In [29]:
cat_surface_depth_nonan.shape
surface_data = cat_surface_depth_nonan[:,:,:5]
sali_depth_use = cat_surface_depth_nonan[:,:,5:]
print(surface_data.shape,sali_depth_use.shape)

(132, 104457, 5) (132, 104457, 33)


In [30]:
available = len(surface_data) 
predict = 1

surface_data_shape = surface_data.shape[1:]
sali_depth_use_shape = sali_depth_use.shape[1:]

N_stack = 6

X_len = available - (N_stack-1)
Y_len = X_len

X = np.zeros(shape = (X_len, N_stack,surface_data_shape[0],surface_data_shape[1]))  
for i in range(X_len):
    X[i] = np.stack(surface_data[i:i+N_stack])

Y = np.zeros(shape = (Y_len,predict,sali_depth_use_shape[0],sali_depth_use_shape[1]))
for i in range(Y_len):
    Y[i] = np.stack(sali_depth_use[i+N_stack-1 : i+N_stack-1+predict])

print(X.shape)
print(Y.shape)

(127, 6, 104457, 5)
(127, 1, 104457, 33)


# 验证集和测试集的划分

In [56]:
test_num = 115
x_train = X[:test_num]
y_train = Y[:test_num]

x_test = X[test_num:]
y_test = Y[test_num:]

(115, 6, 104457, 5) (115, 1, 104457, 33) (12, 6, 104457, 5) (12, 1, 104457, 33)


In [ ]:
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape )

## 修改维度

In [32]:
x_train = np.transpose(x_train, (0,2,1,3))
x_train = x_train.reshape((-1,6,5))

y_train = np.transpose(y_train, (0,2,1,3))
y_train = y_train.reshape((-1,1,33))

x_test = np.transpose(x_test, (0,2,1,3))
x_test = x_test.reshape((-1,6,5))

y_test = np.transpose(y_test, (0,2,1,3))
y_test = y_test.reshape((-1,1,33))
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(12012555, 6, 5) (12012555, 1, 33) (1253484, 6, 5) (1253484, 1, 33)


## 划分训练数据和目标数据

In [33]:
print('x_test.shape:',x_test.shape)
print('y_test.shape',y_test.shape)
print('x_train',x_train.shape)
print('y_train',y_train.shape)

x_test.shape: (1253484, 6, 5)
y_test.shape (1253484, 1, 33)
x_train (12012555, 6, 5)
y_train (12012555, 1, 33)


In [34]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=100)
print(x_train.shape,x_test.shape,x_val.shape,y_train.shape,y_test.shape,y_val.shape)

(8408788, 6, 5) (1253484, 6, 5) (3603767, 6, 5) (8408788, 1, 33) (1253484, 1, 33) (3603767, 1, 33)


# 特征归一化

In [35]:
x_train_shape = x_train.shape
y_train_shape = y_train.shape

x_test_shape = x_test.shape
y_test_shape = y_test.shape

x_val_shape = x_val.shape
y_val_shape = y_val.shape

In [36]:
scaler_f = StandardScaler()
x_train = scaler_f.fit_transform(x_train.reshape(-1,x_train_shape[2])) 
x_test = scaler_f.transform(x_test.reshape(-1,x_test_shape[2]))
x_val = scaler_f.transform(x_val.reshape(-1,x_val_shape[2]))

scaler_l = StandardScaler()
y_train = scaler_l.fit_transform(y_train.reshape(-1,y_train_shape[2])) 
y_test = scaler_l.transform(y_test.reshape(-1,y_test_shape[2])) 
y_val = scaler_l.transform(y_val.reshape(-1,y_val_shape[2])) 

In [37]:
x_train = np.reshape(x_train,(x_train_shape))
x_test = np.reshape(x_test,(x_test_shape))
x_val = np.reshape(x_val,(x_val_shape))
y_train = np.reshape(y_train,(y_train_shape[0],-1))
y_test = np.reshape(y_test,(y_test_shape[0],-1))
y_val = np.reshape(y_val,(y_val_shape[0],-1))

In [38]:
print(x_train.shape,x_test.shape,x_val.shape,y_train.shape,y_test.shape,y_val.shape)

(8408788, 6, 5) (1253484, 6, 5) (3603767, 6, 5) (8408788, 33) (1253484, 33) (3603767, 33)


In [54]:
len(np.unique(x_train))

25509989

# 定义网络结构

In [39]:
class Attention(Layer):
    def __init__(self, units=128, **kwargs):
        self.units = units
        super().__init__(**kwargs)
        
    def __call__(self, inputs):
        """
        Keras的多对一注意力机制。
        @param inputs: 3D tensor with shape (batch_size, time_steps, input_dim).
        @return: 2D tensor with shape (batch_size, 128)
        """
        
        hidden_states = inputs
        hidden_size = int(hidden_states.shape[2])  
        score_first_part = Dense(hidden_size, use_bias=False)(hidden_states) 
        h_t = Lambda(lambda x: x[:, -1, :], output_shape=(hidden_size,))(hidden_states)
        score = Dot(axes=[1, 2])([h_t, score_first_part])  
        attention_weights = Activation('softmax')(score)

        context_vector = Dot(axes=[1, 1])([hidden_states, attention_weights])  
        pre_activation = Concatenate()([context_vector, h_t]) 
        attention_vector = Dense(self.units, use_bias=False, activation='tanh')(pre_activation) 
        
        return attention_vector

    def get_config(self):
        return {'units': self.units}

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [42]:
def my_model():
    
    model_input= Input(shape=(6,5))
   
    model= LSTM(64, return_sequences=True)(model_input)   
    model = Dropout(0.2)(model)
    
    model = LSTM(32, return_sequences=True)(model)
    model = Attention()(model)
    
    model = Flatten()(model)
    
    out = Dense(1)(model)
    model = Model(inputs=model_input, outputs=out)
    return model

In [ ]:
import graphviz
model = my_model()
plot_model(model,show_shapes=True,show_layer_names=True)

## 训练5 - 2000m

In [46]:
def train_model(x_train,y_train,x_val,y_val,name):
    model = my_model()
    model.compile(loss='mse', optimizer='adam')
    checkpoint_path=name
    keras_callbacks   = [
          EarlyStopping(monitor='val_loss', patience=30, mode='min', min_delta=0.001),
          ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
    ]
    model.fit(x_train, y_train, validation_data=(x_val,y_val),
                    epochs=200, batch_size=10240, verbose=2, callbacks=keras_callbacks)

In [48]:
print(x_train.shape,x_test.shape,x_val.shape,y_train.shape,y_test.shape,y_val.shape)

(8408788, 6, 5) (1253484, 6, 5) (3603767, 6, 5) (8408788, 33) (1253484, 33) (3603767, 33)


In [ ]:
depth = [5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,500,600,700,800,900,1000,1100,
              1300,1500,1750,2000] 
for i in range(0,33):
    train_model(x_train,y_train[:,i:i+1],x_val,y_val[:,i:i+1],
                'D:/LSTM/sali/LSTM_sali_5_2000m_'+str(depth[i])+'m.h5')

## 模型评估

In [72]:
import math
def acc(actual, predicted):
    pred_avg = np.average(predicted)

    act_avg = np.average(actual)
    diff_pred = predicted - pred_avg
    diff_act = actual - act_avg
    numerator = np.mean(np.sum(diff_pred*diff_act, axis=0))
    denominator = math.sqrt(np.mean(np.sum(diff_pred**2, axis=0)) * np.mean(np.sum(diff_act**2, axis=0)))
    ret_val = numerator/denominator
    return (100 * ret_val)

def look_loss(history):
    plt.figure(figsize=(6.3, 2.5), dpi = 100)
    plt.plot(history.history['loss'], label='training data')
    plt.plot(history.history['val_loss'], label='validation data')
    plt.title('Loss')
    plt.ylabel('MSE Loss')
    plt.xlabel('Number of epochs')
    plt.grid()
    plt.legend(loc="upper right")
    plt.show()

def all_estimate(best_model,x_test,y_test):
    testPred_1 = best_model.predict(x_test)
    a = y_test
    y_test_p = a.reshape(-1,1)
    testPred_p = testPred_1.reshape(-1,1)

    rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
    print('Test RMSE: %.3f' % rmse)

    print('acc:',acc(y_test_p,testPred_p))

    r2 = r2_score(y_test_p,testPred_p)
    print("R² score:", r2)  
    

def respective_estimate(best_model,x_test,y_test):
    re2_list = []
    rmse_list = []
    for i in range(6):
        testPred = best_model.predict(x_test[i:i+1])
        a = y_test[i:i+1]
        y_test_p = a.reshape(-1,1)
        testPred_p = testPred.reshape(-1,1)
        r2 = r2_score(y_test_p,testPred_p)
        rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
        re2_list.append(r2)
        rmse_list.append(rmse)    
    return re2_list,rmse_list

### 结果评估

In [ ]:
depth = [5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,500,600,700,800,900,1000,1100,
              1300,1500,1750,2000] 
result = [] 
for i in range(0,33):
    best_model = tf.keras.models.load_model('D:/LSTM/sali/LSTM_sali_5_2000m_'+str(depth[i])+'m.h5')
    print('预测第'+str(depth[i])+'m盐度的评价')
    all_estimate(best_model,x_test,y_test[:,i:i+1])
    testPred = best_model.predict(x_test)
    result.append(testPred)

## 反归一化结果评价

In [ ]:
result = np.array(result)
print(result.shape)
result = np.transpose(result,(1,2,0))
print(result.shape)
result = result.reshape(-1,33)
print(result.shape)
y_test = y_test.reshape(-1,33)
print(y_test.shape)

In [ ]:
result_unscaled = scaler_l.inverse_transform(result)
y_test_unscaled = scaler_l.inverse_transform(y_test)
result_unscaled.shape,y_test.shape

In [ ]:
for i in range(0,33):
    print('预测第'+str(depth[i])+'m盐度的评价')
    rmse = sqrt(mean_squared_error(y_test_unscaled[:,i],result_unscaled[:,i]))
    print('Test RMSE: %.3f' % rmse)
    r2 = r2_score(y_test_unscaled[:,i],result_unscaled[:,i])
    print('R² score:',r2)

# 预测结果可视化

## 为归一化之后的预测值和真实值填充nan

In [91]:
cat_surface_depth_test = np.transpose(cat_surface_depth,(1,2,0,3))
print(cat_surface_depth_test.shape)
cat_surface_depth_test = cat_surface_depth_test.reshape ((-1,132*38))
print(cat_surface_depth_test.shape)

(280, 600, 132, 38)
(168000, 5016)


In [95]:
cat_surface_depth_test[cat_surface_depth_test == 32767] = np.nan
print('cat_surface_depth_test删除nan值之前',cat_surface_depth_test.shape)
nan_mask = np.isnan(cat_surface_depth_test)

cat_surface_depth_test删除nan值之前 (168000, 5016)


In [96]:
nan_mask[nan_mask.any(axis=1) == True] = True
nan_masks = nan_mask[:,1]

In [99]:
testPred_with_nan = np.full((168000,12*33), np.nan)  
print(testPred_with_nan.shape)
y_test_with_nan = np.full((168000,12*33), np.nan)  
print(y_test_with_nan.shape)

(168000, 396)
(168000, 396)


In [100]:
result_unscaled_reshape = result_unscaled.reshape(12,-1,33)
print(result_unscaled_reshape.shape)
result_unscaled_reshape = np.transpose(result_unscaled_reshape,(1,0,2))
print(result_unscaled_reshape.shape)
result_unscaled_reshape = result_unscaled_reshape.reshape(-1,12*33)
print(result_unscaled_reshape.shape)

# ======================================== #

y_test_unscaled_reshape = y_test_unscaled.reshape(12,-1,15)
print(y_test_unscaled_reshape.shape)
y_test_unscaled_reshape = np.transpose(y_test_unscaled_reshape,(1,0,2))
print(y_test_unscaled_reshape.shape)
y_test_unscaled_reshape = y_test_unscaled_reshape.reshape(-1,12*33)
print(y_test_unscaled_reshape.shape)

(12, 104729, 33)
(104729, 12, 33)
(104729, 396)
(12, 104729, 33)
(104729, 12, 33)
(104729, 396)


In [102]:
y_test_unscaled.shape, y_test_unscaled_reshape.shape

(1256748, 33)

In [103]:
num = 0
for i,nan_mask in enumerate(nan_masks): 

    if (nan_mask == False):
        testPred_with_nan[i] = result_unscaled_reshape[num]
        y_test_with_nan[i] = y_test_unscaled_reshape[num]
        num = num + 1
        
print(num)

104729


In [104]:
testPred_with_nan = testPred_with_nan.reshape(280, 600,12,33)
testPred_with_nan = np.transpose(testPred_with_nan,(2,3,0,1))

y_test_with_nan = y_test_with_nan.reshape(280, 600,12,33)
y_test_with_nan = np.transpose(y_test_with_nan,(2,3,0,1))

print(testPred_with_nan.shape,y_test_with_nan.shape)

(12, 33, 280, 600) (12, 33, 280, 600)


# 结果可视化

In [105]:
lat = []
lon = []
time = []
lon.append(data_ssh['lon'][:].data)
lat.append(data_ssh['lat'][:].data)

In [107]:
start_year = 2015
end_year =2015
time = []
date_range = pd.date_range(datetime(start_year,1,1),datetime(end_year+1,1,1),freq='1M')
len(date_range)
for i in range(len(date_range)):
    time.append(i)

In [109]:
new_NC = nc.Dataset("D:/LSTM/result_sali_LSTM_5_2000m.nc", 'w', format='NETCDF4')

new_NC.createDimension('lat', len(lat[0]))
new_NC.createDimension('lon', len(lon[0]))
new_NC.createDimension('depth', 33)
new_NC.createDimension('time', len(time))

new_NC.createVariable('lat', 'f', ("lat"))
new_NC.createVariable('lon', 'f', ("lon"))
new_NC.createVariable('depth', 'f', ("depth"))
new_NC.createVariable('true_ss', 'f',("time","depth","lat","lon"))
new_NC.createVariable('pred_ss', 'f',("time","depth","lat","lon"))

time_var = new_NC.createVariable('time', 'f4',("time"))
time_var.units = 'months since 2015-01-15'
time_var.long_name = 'Months in Monthly Means'
time_var.axis = 'T'

new_NC.variables['lat'][:] = lat[0]
new_NC.variables['lon'][:] = lon[0]
new_NC.variables['time'][:] = np.array(time)
new_NC.variables['depth'][:] = depth

new_NC.variables['true_ss'][:]=np.array(y_test_with_nan)
new_NC.variables['pred_ss'][:]=np.array(testPred_with_nan)

new_NC.close()